<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据预处理-(判定贷款用户是否逾期)" data-toc-modified-id="数据预处理-(判定贷款用户是否逾期)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据预处理 (判定贷款用户是否逾期)</a></span><ul class="toc-item"><li><span><a href="#删除无用特征" data-toc-modified-id="删除无用特征-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>删除无用特征</a></span></li><li><span><a href="#数据格式化---X_date" data-toc-modified-id="数据格式化---X_date-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>数据格式化 - X_date</a></span></li><li><span><a href="#数据处理---类别特征-X_cate" data-toc-modified-id="数据处理---类别特征-X_cate-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>数据处理 - 类别特征 X_cate</a></span></li><li><span><a href="#数据处理---其他非数值型特征" data-toc-modified-id="数据处理---其他非数值型特征-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>数据处理 - 其他非数值型特征</a></span></li><li><span><a href="#数据处理---数值型特征-X_num" data-toc-modified-id="数据处理---数值型特征-X_num-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>数据处理 - 数值型特征 X_num</a></span></li><li><span><a href="#特征拼接与存储" data-toc-modified-id="特征拼接与存储-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>特征拼接与存储</a></span></li></ul></li><li><span><a href="#遇到的问题" data-toc-modified-id="遇到的问题-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>遇到的问题</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# 数据预处理 (判定贷款用户是否逾期)

给定金融数据，预测贷款用户是否会逾期。
（status是标签：0表示未逾期，1表示逾期。）

**Task5（特征工程1）** - 数据类型转换, 无用特征删除, 缺失值处理（尝试不同的填充看效果）及数据探索

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 数据集预览
data = pd.read_csv('data.csv')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()

(4754, 90)


,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [3]:
y = data.status
X = data.drop('status', axis=1)

# 观测正负样本是否均衡
y.value_counts()

0    3561
1    1193
Name: status, dtype: int64

In [4]:
# 变量类型
set(X.dtypes)

{dtype('int64'), dtype('float64'), dtype('O')}

## 删除无用特征

**删除原则**

> 1）属性值单一的特征;

> 2）观测特征取值以及label意义, 挑选和预测无关的特征

1）**属性值单一的特征** - 'bank_card_no','source'的取值**无区分度**

In [5]:
for col in X.columns:
    if len(X[col].unique()) == 1:
        print(col, X[col].unique())
        X.drop(col, axis = 1, inplace = True)

bank_card_no ['卡号1']
source ['xs']


2）观测特征取值以及label意义, 是否**和预测无关** - 'Unnamed: 0', 'custid', 'trade_no'和id_name'**与预测无关**

In [6]:
for col in X.columns:
    cnt = X[col].count()    # 没有统计缺失值
    if len(list(X[col].unique())) in [cnt, cnt+1]:
        print(col)

Unnamed: 0
custid
trade_no


In [7]:
# X['Unnamed: 0']
# X['custid']
# X['trade_no]
# X['id_name'].value_counts()

In [8]:
X.drop(['Unnamed: 0', 'custid', 'trade_no', 'id_name'], axis=1, inplace=True)

## 数据格式化 - X_date

（此处主要针对**时间特征**, 且此处时间特征仅包含日期特征）

**日期特征处理流程**

> 1）浮点型日期转换成字符串型

> 2）取出日期，构建年份、月份、周几等特征

> 3）进一步（特征构建）：groupby对特征进行统计分析

In [9]:
dateFeatures = ['first_transaction_time', 'latest_query_time', 'loans_latest_time']
X_date = X[dateFeatures]

In [10]:
X_date.head()

,first_transaction_time,latest_query_time,loans_latest_time
0,20130817.0,2018-04-25,2018-04-19
1,20160402.0,2018-05-03,2018-05-05
2,20170617.0,2018-05-05,2018-05-01
3,20130516.0,2018-05-05,2018-05-03
4,20170312.0,2018-04-15,2018-01-07


1）浮点型日期转换成字符串型

In [11]:
# 首先填充缺失值
X_date['first_transaction_time'].fillna(X_date['first_transaction_time'].median(), inplace = True)
# 转成字符串型日期
X_date['first_transaction_time'] = X_date['first_transaction_time'].apply(lambda x:str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8])

2） 提取特征：年份、月份、星期几

In [12]:
X_date['first_transaction_time_year'] = pd.to_datetime(X_date['first_transaction_time']).dt.year
X_date['first_transaction_time_month'] = pd.to_datetime(X_date['first_transaction_time']).dt.month
X_date['first_transaction_time_weekday'] = pd.to_datetime(X_date['first_transaction_time']).dt.weekday

X_date['latest_query_time_year'] = pd.to_datetime(X_date['latest_query_time']).dt.year
X_date['latest_query_time_month'] = pd.to_datetime(X_date['latest_query_time']).dt.month
X_date['latest_query_time_weekday'] = pd.to_datetime(X_date['latest_query_time']).dt.weekday

X_date['loans_latest_time_year'] = pd.to_datetime(X_date['loans_latest_time']).dt.year
X_date['loans_latest_time_month'] = pd.to_datetime(X_date['loans_latest_time']).dt.month
X_date['loans_latest_time_weekday'] = pd.to_datetime(X_date['loans_latest_time']).dt.weekday

In [13]:
# 填充缺失值
X_date['latest_query_time_year'].fillna(X_date['latest_query_time_year'].median(), inplace = True)
X_date['latest_query_time_month'].fillna(X_date['latest_query_time_month'].median(), inplace = True)
X_date['latest_query_time_weekday'].fillna(X_date['latest_query_time_weekday'].median(), inplace = True)

X_date['loans_latest_time_year'].fillna(X_date['loans_latest_time_year'].median(), inplace = True)
X_date['loans_latest_time_month'].fillna(X_date['loans_latest_time_month'].median(), inplace = True)
X_date['loans_latest_time_weekday'].fillna(X_date['loans_latest_time_weekday'].median(), inplace = True)

In [14]:
X_date.drop(dateFeatures, axis = 1, inplace=True)

## 数据处理 - 类别特征 X_cate

> 1）字符型类别特征编码

> 2）缺失值填充

> 3）类别特征Label与One-Hot编码

**类别特征缺失值填充常用方法**：分箱处理（单独填充为一个类别）、众数填充

观察取值和属性名称, 挑选类别特征

In [15]:
for col in X:
    cnt = len(X[col].unique())
    if cnt < 15:
        print(col, cnt, X[col].unique())

regional_mobility 6 [ 3.  4.  1.  2.  5. nan]
student_feature 3 [nan  1.  2.]
is_high_user 2 [0 1]
avg_consume_less_12_valid_month 13 [ 7.  5.  6.  8.  9.  3.  4. 11. 10.  0.  2.  1. nan]
top_trans_count_last_1_month 9 [0.15 0.05 0.65 1.   0.1  0.3  0.4  0.2   nan]
reg_preference_for_trad 6 ['一线城市' '三线城市' '境外' '二线城市' '其他城市' nan]
railway_consume_count_last_12_month 7 [ 0.  1.  2.  4. nan  3. 30.]
jewelry_consume_count_last_6_month 8 [ 0.  1. nan  2.  6.  3.  4.  5.]


In [16]:
categoryFeatures = ['regional_mobility', 'student_feature', 'is_high_user', 'avg_consume_less_12_valid_month', 'reg_preference_for_trad']
X_cate = X[categoryFeatures]

1）字符型类别特征编码

In [17]:
dic = {}
for i, val in enumerate(list(X_cate['reg_preference_for_trad'].unique())):
    dic[val] = i

X_cate['reg_preference_for_trad'] = X_cate['reg_preference_for_trad'].map(dic)

In [18]:
X_cate.describe().T\
    .assign(missing_pct=data.apply(lambda x : (len(x)-x.count())/len(x)))

,count,mean,std,min,25%,50%,75%,max,missing_pct
regional_mobility,4752.0,2.678662,0.890360,1.0,2.0,3.0,3.0,5.0,0.000421
student_feature,1756.0,1.001139,0.033739,1.0,1.0,1.0,1.0,2.0,0.630627
is_high_user,4754.0,0.011149,0.105007,0.0,0.0,0.0,0.0,1.0,0.000000
avg_consume_less_12_valid_month,4752.0,6.572601,1.390723,0.0,6.0,7.0,7.0,11.0,0.000421
reg_preference_for_trad,4754.0,0.375053,0.693858,0.0,0.0,0.0,1.0,5.0,0.000421


2）缺失值填充：单独填充为一个类别/众数填充

student_feature 缺失占比一般以上, 将其缺失值**单独填充为1个类别**（用-1填充）。 

In [19]:
X_cate['student_feature'].value_counts()

1.0    1754
2.0       2
Name: student_feature, dtype: int64

In [20]:
X_cate['student_feature'].fillna(-1, inplace = True)

In [21]:
X_cate['student_feature'].value_counts()

-1.0    2998
 1.0    1754
 2.0       2
Name: student_feature, dtype: int64

其他特征缺失值数目较少, 考虑用众数填充

In [22]:
for col in X_cate.columns:
    summ = X_cate[col].isnull().sum()
    if summ:
        X_cate[col].fillna(X_cate[col].mode()[0], inplace = True)

3）类别特征Label与One-Hot编码

|编码方法|介绍|优点|缺点|适用场景|
|--|--|--|--|--|
|label encoding|将类别变量中每一类别赋一数值，从而转换成数值型|略|赋值没有数值意义||
|One-hot encoding|它的值只有0/1，不同的类型存储在垂直的空间|xx|当类别的数量很多时，特征空间会变得非常大|①除了树模型都要One-hot,因为label没有数值意义|

## 数据处理 - 其他非数值型特征

> 没有额外的待处理非数字型特征

In [23]:
X_str = X.select_dtypes(include=['O']).copy()    # 不是复制视图, 所以加copy()

In [24]:
X_str.head()

,reg_preference_for_trad,latest_query_time,loans_latest_time
0,一线城市,2018-04-25,2018-04-19
1,一线城市,2018-05-03,2018-05-05
2,一线城市,2018-05-05,2018-05-01
3,三线城市,2018-05-05,2018-05-03
4,一线城市,2018-04-15,2018-01-07


## 数据处理 - 数值型特征 X_num

> 1）缺失值处理

> 2）去掉取值变化小的特征：统计各个列标准差，将标准差小于0.1的特征剔除

**连续特征缺失值填充常用方法**：中位数填充，平均数一般不用的(均值受极端值影响太大)。

In [25]:
X_num = X.select_dtypes(exclude=['O']).copy()    # 不是复制视图, 所以加copy()
X_num.shape

(4754, 80)

In [26]:
for col in X_num.columns:
    if col in dateFeatures + categoryFeatures:
        print(col)
        X_num.drop(col, axis = 1, inplace = True)
X_num.shape

regional_mobility
student_feature
is_high_user
first_transaction_time
avg_consume_less_12_valid_month


(4754, 75)

In [27]:
(X_num.describe().T
 .drop(['25%','50%','75%'],axis=1)
 .assign(missing_pct=data.apply(lambda x: (len(x)-x.count())/len(x)))).T

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,repayment_capability,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4752.000000,4752.000000,4754.000000,4751.000000,4752.000000,4752.000000,4752.000000,4746.000000,4752.000000,4.754000e+03,...,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4450.000000,4457.000000
mean,0.021806,0.901294,1940.197728,14.160674,0.804411,0.365425,17.502946,29.029920,21.751263,1.870201e+04,...,3390.038142,1820.357864,9187.009199,76.042630,4.732331,5.227507,16153.690823,8007.696881,24.112809,55.181512
std,0.041527,0.144856,3923.971494,694.180473,0.196920,0.170196,4.475616,22.722432,16.474916,5.221783e+04,...,1474.206546,583.418291,7371.257043,14.536819,2.974596,3.409292,14301.037628,5679.418585,37.725724,53.486408
min,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,4.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
max,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,234.000000,2.459390e+06,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000
missing_pct,0.000421,0.000421,0.000000,0.000631,0.000421,0.000421,0.000421,0.001683,0.000421,0.000000e+00,...,0.062474,0.062474,0.062474,0.062474,0.062474,0.062474,0.062474,0.062474,0.063946,0.062474


1）缺失值处理

主要填充方法采用众数、中位数和模型填充等，**平均数一般不用的(均值受极端值影响太大)**。

In [28]:
# 统计各列缺失值的比例
col_missing = {}
for col in X_num.columns:
    summ = X_num[col].isnull().sum()
    if summ:
        col_missing[col] = float('%.4f'%(summ*100/len(data)))
    
col_missing = sorted(col_missing.items(), key = lambda d:-d[1])
for col, rate in col_missing[:10]:
    print(rate, '%', col)

8.9609 % cross_consume_count_last_1_month
6.3946 % apply_score
6.3946 % apply_credibility
6.3946 % query_org_count
6.3946 % query_finance_count
6.3946 % query_cash_count
6.3946 % query_sum_count
6.3946 % latest_one_month_apply
6.3946 % latest_three_month_apply
6.3946 % latest_six_month_apply


缺失特征用**中位数**填充。

In [29]:
for col in X_num.columns:
    summ = X_num[col].isnull().sum()
    if summ:
        X_num[col].fillna(X_num[col].median(), inplace = True)

2）去掉取值变化小的特征

In [30]:
for col in X_num.columns:
    rate = X_num[col].std()
    if rate < 0.1:
        print(col, rate)
        X_num.drop(col, axis = 1, inplace = True)

low_volume_percent 0.04151908997545957
avg_price_top_last_12_valid_month 0.0993155786043876


3） 归一化

> 归一化, 可以加快梯度下降求最优解的速度, 也有利于提高精度

In [31]:
# 最大最小归一化
# X_num = X_num.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

## 特征拼接与存储

In [32]:
X = pd.concat([X_cate, X_date, X_num], axis=1)

In [33]:
X.shape

(4754, 87)

In [34]:
import pickle
with open('feature.pkl', 'wb') as f:
    pickle.dump(X, f)

# 遇到的问题

> 特征数目和行数相同, 肯定和预测无关吗？ 如果是属于类似连续型特征呢？

# Reference

1）[特征工程处理](http://cking0821.club/2018/05/31/%E7%89%B9%E5%BE%81%E5%B7%A5%E7%A8%8B%E5%A4%84%E7%90%86/)

2）[数据分析工具之Python大法（一）](https://zhuanlan.zhihu.com/p/21401793)

3）[机器学习-Label Encoding与One Hot的区别-20180513](https://zhuanlan.zhihu.com/p/36804348)